In [73]:
import requests 
import os
import logging
import sys
# from telegram.error import BadRequest, NetworkError, TelegramError

# use log file to store error messages
logging.basicConfig(filename='error_test.log', level=logging.INFO,
                    format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p')


telegram_chat_id = os.environ.get('telegram_chat_id')
telegramToken = os.environ.get('telegramToken1d') 

def telegram_send_message():    
    lmsg = "telegram_send_message"

    params = {
    "chat_id": telegram_chat_id,
    "text": lmsg,
    "parse_mode": "HTML2",
    }
    try:
        resp = requests.post("https://api.telegram.org/bot{}/sendMessage".format(telegramToken), params=params)
        resp.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        msg = sys._getframe(  ).f_code.co_name+" - An Http Error occurred:" + repr(errh)
        print(msg)
        logging.exception(msg)
    except requests.exceptions.ConnectionError as errc:
        msg = sys._getframe(  ).f_code.co_name+" - An Error Connecting to the API occurred:" + repr(errc)
        print(msg)
        logging.exception(msg)
    except requests.exceptions.Timeout as errt:
        msg = sys._getframe(  ).f_code.co_name+" - A Timeout Error occurred:" + repr(errt)
        print(msg)
        logging.exception(msg)
    except requests.exceptions.RequestException as err:
        msg = sys._getframe(  ).f_code.co_name+" - An Unknown Error occurred" + repr(err)
        print(msg)
        logging.exception(msg)

In [74]:
telegram_send_message()

telegram_send_message - An Http Error occurred:HTTPError('400 Client Error: Bad Request for url: https://api.telegram.org/bot5791916657:AAF7DPD1tkco53NOlj0fLY7Jg43EHBZWLME/sendMessage?chat_id=1144450791&text=telegram_send_message&parse_mode=HTML2')


In [1]:
def test_message(msg):
    print(msg)
    msg = "nova mensagem"
    print(msg)


In [2]:
test_message("novo teste")

novo teste
nova mensagem


In [9]:
import pandas as pd
df_open_positions_1h = pd.read_csv('positions1h.csv')
df_open_positions_1h = df_open_positions_1h[df_open_positions_1h.position == 1].Currency
df_open_positions_1h

0     ANTBUSD
2     CVXBUSD
3     ENSBUSD
5     FTMBUSD
6    MASKBUSD
8     XMRBUSD
Name: Currency, dtype: object

In [10]:
df = df_open_positions_1h.reset_index(drop=True)
df

0     ANTBUSD
1     CVXBUSD
2     ENSBUSD
3     FTMBUSD
4    MASKBUSD
5     XMRBUSD
Name: Currency, dtype: object